# Exp R3a: Spinodal Estimation (Ordered → Collapse)

## 目的
Ordered状態から開始し、λを上げていき「崩壊イベント」が起きるλ_↑（上側スピノーダル）を測定。

## 実験設計
- **初期状態**: ordered-init（λ=0.35で30 epoch学習済み）
- **Sweep方向**: λ↑（ordered → collapse方向）
- **イベント判定**: test error > 40% が数エポック継続
- **保持条件**: short (1 epoch), long (3 epochs)
- **η**: 0.4, 0.8
- **Seeds**: 20
- **Total**: 2η × 2保持 × 20seeds = 80 runs (分割)

## このノートブック
- **条件**: 全条件の半分（seeds 0-9）
- **runs**: 40
- **推定時間**: ~7時間

## 関連ノートブック
- R3a (this): ordered→collapse (spinodal λ_↑)
- R3b: collapse→ordered (spinodal λ_↓)

In [ ]:
# ===== セットアップ =====
from google.colab import drive
drive.mount('/content/drive')

import os
import glob
from datetime import datetime

EXP_NAME = 'exp_R3_spinodal'
NOTEBOOK_ID = 'R3a'
BASE_DIR = '/content/drive/MyDrive/dual-gradient-learning/Paper-A'

existing = glob.glob(f'{BASE_DIR}/{EXP_NAME}_*')
if existing:
    SAVE_DIR = sorted(existing)[-1]
    print(f'🔄 Resuming from: {SAVE_DIR}')
else:
    TIMESTAMP = datetime.now().strftime('%Y%m%d_%H%M%S')
    SAVE_DIR = f'{BASE_DIR}/{EXP_NAME}_{TIMESTAMP}'
    os.makedirs(SAVE_DIR, exist_ok=True)
    print(f'🆕 New experiment: {SAVE_DIR}')

os.makedirs(f'{SAVE_DIR}/figures', exist_ok=True)
os.makedirs(f'{SAVE_DIR}/checkpoints', exist_ok=True)
print(f'Save directory: {SAVE_DIR}')
print(f'This notebook: {NOTEBOOK_ID} (ordered→collapse)')

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader, Dataset
from torch.nn.utils import parameters_to_vector
import torchvision
import torchvision.transforms as transforms
from torchvision.models import resnet18
import numpy as np
import json
import time

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'Device: {device}')
if torch.cuda.is_available():
    print(f'GPU: {torch.cuda.get_device_name()}')

In [ ]:
# ===== ResNet18 =====
def get_resnet18():
    model = resnet18(weights=None, num_classes=10)
    model.conv1 = nn.Conv2d(3, 64, kernel_size=3, stride=1, padding=1, bias=False)
    model.maxpool = nn.Identity()
    return model

class IndexedDataset(Dataset):
    def __init__(self, dataset):
        self.dataset = dataset
    def __getitem__(self, idx):
        img, label = self.dataset[idx]
        return img, label, idx
    def __len__(self):
        return len(self.dataset)

In [ ]:
# ===== 実験パラメータ =====
BATCH_SIZE = 256
NUM_WORKERS = 4
LR = 0.1
K = 16

# Spinodal estimation parameters
INIT_LAMBDA = 0.35  # ordered regime
INIT_EPOCHS = 30    # epochs to establish ordered state
COLLAPSE_THRESHOLD = 0.40  # error threshold for collapse
COLLAPSE_SUSTAIN = 3  # epochs to sustain for confirmation
MAX_SWEEP_EPOCHS = 100  # max epochs during sweep

# λ sweep range (ordered → collapse)
LAMBDA_START = 0.40
LAMBDA_END = 0.70
LAMBDA_STEP = 0.02

# Conditions for this notebook
NOISE_RATES = [0.4, 0.8]
HOLD_TYPES = ['short', 'long']  # 1 epoch vs 3 epochs per λ step
SEEDS = list(range(10))  # seeds 0-9 (R3a), seeds 10-19 would be in another nb if needed

experiments = []
for eta in NOISE_RATES:
    for hold_type in HOLD_TYPES:
        for seed in SEEDS:
            experiments.append({
                'noise_rate': eta,
                'hold_type': hold_type,
                'hold_epochs': 1 if hold_type == 'short' else 3,
                'seed': seed,
                'direction': 'up'  # ordered → collapse
            })

print(f'Total experiments: {len(experiments)}')
print(f'Noise rates: {NOISE_RATES}')
print(f'Hold types: {HOLD_TYPES}')
print(f'Seeds: {SEEDS}')

config = {
    'experiment': EXP_NAME,
    'notebook_id': NOTEBOOK_ID,
    'direction': 'ordered_to_collapse',
    'parameters': {
        'init_lambda': INIT_LAMBDA,
        'init_epochs': INIT_EPOCHS,
        'collapse_threshold': COLLAPSE_THRESHOLD,
        'collapse_sustain': COLLAPSE_SUSTAIN,
        'lambda_range': [LAMBDA_START, LAMBDA_END, LAMBDA_STEP],
        'noise_rates': NOISE_RATES,
        'hold_types': HOLD_TYPES,
        'seeds': SEEDS
    }
}
with open(f'{SAVE_DIR}/{NOTEBOOK_ID}_config.json', 'w') as f:
    json.dump(config, f, indent=2)

In [ ]:
# ===== ユーティリティ関数 =====
def set_seed(seed):
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    np.random.seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

def inject_label_noise(labels, noise_rate, seed):
    np.random.seed(seed)
    noisy_labels = labels.copy()
    n_samples = len(labels)
    n_noisy = int(noise_rate * n_samples)
    noisy_indices = np.random.choice(n_samples, n_noisy, replace=False)
    for idx in noisy_indices:
        original = labels[idx]
        new_label = np.random.choice([l for l in range(10) if l != original])
        noisy_labels[idx] = new_label
    return noisy_labels

def load_cifar10():
    transform_train = transforms.Compose([
        transforms.RandomCrop(32, padding=4),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
    ])
    transform_test = transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
    ])
    trainset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform_train)
    testset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform_test)
    return trainset, testset

def get_data_loaders(trainset, testset):
    train_loader = DataLoader(IndexedDataset(trainset), batch_size=BATCH_SIZE, shuffle=True, num_workers=NUM_WORKERS, pin_memory=True)
    test_loader = DataLoader(testset, batch_size=BATCH_SIZE, shuffle=False, num_workers=NUM_WORKERS, pin_memory=True)
    return train_loader, test_loader

def evaluate(model, test_loader):
    model.eval()
    correct, total = 0, 0
    with torch.no_grad():
        for inputs, targets in test_loader:
            inputs, targets = inputs.to(device), targets.to(device)
            outputs = model(inputs)
            _, predicted = outputs.max(1)
            total += targets.size(0)
            correct += predicted.eq(targets).sum().item()
    return correct / total

In [ ]:
# ===== Training functions =====
def train_one_epoch(model, optimizer, train_loader, clean_labels_tensor, noisy_labels_tensor, lam, cached_g_value, global_step):
    """Train for one epoch and return updated cached_g_value and global_step"""
    criterion = nn.CrossEntropyLoss()
    model.train()
    epoch_cos = []
    
    for inputs, _, indices in train_loader:
        inputs = inputs.to(device, non_blocking=True)
        indices = indices.to(device, non_blocking=True)
        batch_noisy = noisy_labels_tensor[indices]
        batch_clean = clean_labels_tensor[indices]
        
        optimizer.zero_grad()
        outputs = model(inputs)
        loss_struct = criterion(outputs, batch_noisy)
        loss_struct.backward(retain_graph=True)
        g_struct = parameters_to_vector([p.grad for p in model.parameters()]).clone()
        
        if global_step % K == 0 or cached_g_value is None:
            optimizer.zero_grad()
            outputs = model(inputs)
            loss_value = criterion(outputs, batch_clean)
            loss_value.backward()
            cached_g_value = parameters_to_vector([p.grad for p in model.parameters()]).clone()
        
        g_struct_norm = g_struct / (g_struct.norm() + 1e-12)
        g_value_norm = cached_g_value / (cached_g_value.norm() + 1e-12)
        
        cos_sim = (g_struct_norm @ g_value_norm).item()
        epoch_cos.append(cos_sim)
        
        g_mix = (1 - lam) * g_struct_norm + lam * g_value_norm
        
        optimizer.zero_grad()
        idx = 0
        for p in model.parameters():
            numel = p.numel()
            p.grad = g_mix[idx:idx+numel].view(p.shape).clone()
            idx += numel
        optimizer.step()
        global_step += 1
    
    return cached_g_value, global_step, np.mean(epoch_cos)

def find_spinodal_up(model, optimizer, train_loader, test_loader, clean_labels_tensor, noisy_labels_tensor, hold_epochs):
    """
    Find λ_↑ (collapse spinodal) by sweeping λ upward from ordered state.
    Returns the λ at which collapse is detected.
    """
    cached_g_value = None
    global_step = 0
    global_epoch = 0
    
    history = []
    collapse_counter = 0
    collapse_lambda = None
    
    # Phase 1: Initialize in ordered state
    print(f'    Phase 1: Establishing ordered state at λ={INIT_LAMBDA}')
    for ep in range(INIT_EPOCHS):
        cached_g_value, global_step, avg_cos = train_one_epoch(
            model, optimizer, train_loader, clean_labels_tensor, noisy_labels_tensor,
            INIT_LAMBDA, cached_g_value, global_step
        )
        global_epoch += 1
        
        if (ep + 1) % 10 == 0:
            acc = evaluate(model, test_loader)
            print(f'      Epoch {ep+1}: error={1-acc:.4f}')
    
    init_acc = evaluate(model, test_loader)
    print(f'    Initial state: error={1-init_acc:.4f}')
    
    # Phase 2: Sweep λ upward
    print(f'    Phase 2: Sweeping λ from {LAMBDA_START} to {LAMBDA_END}')
    current_lambda = LAMBDA_START
    
    while current_lambda <= LAMBDA_END:
        # Train at current λ for hold_epochs
        for h in range(hold_epochs):
            cached_g_value, global_step, avg_cos = train_one_epoch(
                model, optimizer, train_loader, clean_labels_tensor, noisy_labels_tensor,
                current_lambda, cached_g_value, global_step
            )
            global_epoch += 1
        
        # Evaluate
        test_acc = evaluate(model, test_loader)
        test_error = 1 - test_acc
        
        history.append({
            'lambda': current_lambda,
            'test_error': test_error,
            'global_epoch': global_epoch
        })
        
        # Check for collapse
        if test_error > COLLAPSE_THRESHOLD:
            collapse_counter += 1
            if collapse_counter >= COLLAPSE_SUSTAIN:
                collapse_lambda = current_lambda
                print(f'    ⚡ COLLAPSE detected at λ={collapse_lambda:.3f} (error={test_error:.4f})')
                break
        else:
            collapse_counter = 0
        
        current_lambda += LAMBDA_STEP
        current_lambda = round(current_lambda, 3)
    
    if collapse_lambda is None:
        collapse_lambda = LAMBDA_END  # Censored
        print(f'    No collapse within range (censored at λ={LAMBDA_END})')
    
    return collapse_lambda, history, init_acc

In [ ]:
# ===== データ準備 =====
trainset, testset = load_cifar10()
clean_labels = np.array(trainset.targets)
train_loader, test_loader = get_data_loaders(trainset, testset)
print('Data prepared')

warmup_model = get_resnet18().to(device)
for _ in range(20):
    _ = warmup_model(torch.randn(BATCH_SIZE, 3, 32, 32, device=device))
del warmup_model
torch.cuda.empty_cache()
print('GPU warmed up')

In [ ]:
# ===== メイン実験ループ =====
results = []
checkpoint_file = f'{SAVE_DIR}/{NOTEBOOK_ID}_checkpoint.json'
completed = set()

if os.path.exists(checkpoint_file):
    with open(checkpoint_file, 'r') as f:
        results = json.load(f)
    for r in results:
        completed.add((r['noise_rate'], r['hold_type'], r['seed']))
    print(f'Checkpoint loaded: {len(results)} runs completed')

total = len(experiments)

for exp in experiments:
    eta = exp['noise_rate']
    hold_type = exp['hold_type']
    hold_epochs = exp['hold_epochs']
    seed = exp['seed']
    
    if (eta, hold_type, seed) in completed:
        continue
    
    run_num = len(completed) + 1
    print(f'\n[{run_num}/{total}] η={eta} hold={hold_type} seed={seed}')
    
    set_seed(seed)
    noisy_labels = inject_label_noise(clean_labels, eta, seed)
    clean_labels_tensor = torch.tensor(clean_labels, device=device)
    noisy_labels_tensor = torch.tensor(noisy_labels, device=device)
    
    model = get_resnet18().to(device)
    optimizer = optim.SGD(model.parameters(), lr=LR, momentum=0.9, weight_decay=5e-4)
    
    start_time = time.time()
    collapse_lambda, history, init_acc = find_spinodal_up(
        model, optimizer, train_loader, test_loader,
        clean_labels_tensor, noisy_labels_tensor, hold_epochs
    )
    elapsed = time.time() - start_time
    
    result = {
        'experiment_id': f'{NOTEBOOK_ID}-{run_num:03d}',
        'experiment': EXP_NAME,
        'notebook_id': NOTEBOOK_ID,
        'direction': 'up',
        'noise_rate': eta,
        'hold_type': hold_type,
        'hold_epochs': hold_epochs,
        'seed': seed,
        'spinodal_lambda': collapse_lambda,
        'init_error': 1 - init_acc,
        'time_seconds': elapsed,
        'history': history
    }
    results.append(result)
    completed.add((eta, hold_type, seed))
    
    print(f'  λ_↑ = {collapse_lambda:.3f} | Time: {elapsed/60:.1f} min')
    
    with open(checkpoint_file, 'w') as f:
        json.dump(results, f, indent=2)
    
    remaining = total - run_num
    eta_hours = remaining * elapsed / 3600
    print(f'  Progress: {run_num}/{total} | ETA: {eta_hours:.1f} hours')
    
    del model, optimizer
    torch.cuda.empty_cache()

print('\n' + '='*70)
print(f'{NOTEBOOK_ID} COMPLETED!')
print('='*70)

In [ ]:
# ===== 結果保存と分析 =====
import pandas as pd
import matplotlib.pyplot as plt

with open(f'{SAVE_DIR}/{NOTEBOOK_ID}_results.json', 'w') as f:
    json.dump(results, f, indent=2)

df = pd.DataFrame([{k: v for k, v in r.items() if k != 'history'} for r in results])
df.to_csv(f'{SAVE_DIR}/{NOTEBOOK_ID}_results.csv', index=False)

print('\n--- Spinodal λ_↑ Summary ---')
for eta in NOISE_RATES:
    for hold_type in HOLD_TYPES:
        subset = df[(df['noise_rate'] == eta) & (df['hold_type'] == hold_type)]
        if len(subset) > 0:
            mean_sp = subset['spinodal_lambda'].mean()
            std_sp = subset['spinodal_lambda'].std()
            print(f'  η={eta}, {hold_type}: λ_↑ = {mean_sp:.3f} ± {std_sp:.3f}')

# Visualization
fig, axes = plt.subplots(1, 2, figsize=(12, 5))

for i, eta in enumerate(NOISE_RATES):
    ax = axes[i]
    for hold_type, color in [('short', 'blue'), ('long', 'red')]:
        subset = df[(df['noise_rate'] == eta) & (df['hold_type'] == hold_type)]
        if len(subset) > 0:
            ax.scatter(subset['seed'], subset['spinodal_lambda'], 
                      c=color, label=hold_type, alpha=0.7, s=50)
            ax.axhline(subset['spinodal_lambda'].mean(), color=color, linestyle='--', alpha=0.5)
    
    ax.set_xlabel('Seed', fontsize=12)
    ax.set_ylabel('λ_↑ (Collapse Spinodal)', fontsize=12)
    ax.set_title(f'η={eta}: Collapse Spinodal Distribution', fontsize=12)
    ax.legend()
    ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig(f'{SAVE_DIR}/figures/{NOTEBOOK_ID}_spinodal_distribution.png', dpi=150, bbox_inches='tight')
plt.show()

print(f'\nResults saved to {SAVE_DIR}')
print('Next: Run R3b for recovery spinodal (λ_↓)')